In [1]:
import pandas as pd
import numpy as np

In [2]:
auctions = pd.read_csv("auctions.csv", dtype={"ref_type_id": np.uint8, "source_id": np.uint8})

In [3]:
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [4]:
auctions_sorted = auctions.sort_values(["device_id","date"], ascending = True)

In [5]:
auctions_sorted.head()

,date,device_id,ref_type_id,source_id
36601349,2019-04-23 15:00:57.222979,40621409780134,1,0
30900602,2019-04-19 19:40:28.465866,41863526108385,1,8
14875535,2019-04-20 02:52:26.892880,41863526108385,1,3
32062481,2019-04-20 02:59:02.509230,41863526108385,1,3
45613542,2019-04-20 03:06:01.675788,41863526108385,1,3


In [9]:
window_18to20 = auctions.loc[(auctions['date'].dt.day > 17) & (auctions['date'].dt.day < 21)]

In [20]:
window_18to20.loc[auctions['device_id'] == 186034136943920]

,date,device_id,ref_type_id,source_id
4859451,2019-04-18 19:23:42.206032,186034136943920,1,1
12258812,2019-04-20 03:17:51.945332,186034136943920,1,1
12710445,2019-04-20 03:20:06.814613,186034136943920,1,1
16127540,2019-04-18 16:42:46.331894,186034136943920,1,1
25704121,2019-04-20 23:34:37.264901,186034136943920,1,1
25757742,2019-04-20 03:18:31.499631,186034136943920,1,1
43476239,2019-04-20 03:18:13.985766,186034136943920,1,1


In [23]:
real_label = window_18to20.groupby(['device_id']).agg({'date': 'min'})

In [24]:
real_label['date'] = real_label['date']

,date
device_id,
41863526108385,2019-04-19 19:40:28.465866
135153013040192,2019-04-20 04:10:54.009137
161514654074162,2019-04-18 02:52:46.357746
181891380775191,2019-04-20 23:19:25.420614
186034136943920,2019-04-18 16:42:46.331894
283297668933729,2019-04-18 22:24:44.520583
345999128501141,2019-04-18 15:04:50.380240
360710529886978,2019-04-18 16:16:12.637969
365882020742330,2019-04-19 01:51:00.134292


In [6]:
auctions_sorted['timediff'] = np.where(auctions_sorted['device_id'] == auctions_sorted['device_id'].shift(), auctions_sorted['date'] - auctions_sorted['date'].shift(1), np.nan)

In [7]:
auctions_sorted['time_to_reappear'] = auctions_sorted['timediff'].dt.total_seconds()

In [10]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    return df_sorted

In [9]:
auctions_sorted.head()

,device_id,date,timediff,time_to_reappear
36601349,40621409780134,2019-04-23 15:00:57.222979,NaT,NaN
30900602,41863526108385,2019-04-19 19:40:28.465866,NaT,NaN
14875535,41863526108385,2019-04-20 02:52:26.892880,07:11:58.427014,25918.427014
32062481,41863526108385,2019-04-20 02:59:02.509230,00:06:35.616350,395.616350
45613542,41863526108385,2019-04-20 03:06:01.675788,00:06:59.166558,419.166558
